## **Setup**

In [1]:
from fastai.vision.all import *

In [1]:
import os
import shutil
import glob
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pytz
from zipfile import ZipFile
from tempfile import TemporaryDirectory
from PIL import Image

from sklearn.metrics import f1_score, accuracy_score

# plot options
# plt.rcParams.update({'font.size': 11})
plt.style.use('fivethirtyeight')

# **Functions**



In [2]:
# def display_mask(imgnum):
#     img=Image.open('/content/All_Data/Labels/train_gt/image_'+imgnum+'.png')
#     img=np.asarray(img)*255
#     display(Image.fromarray(np.uint8(img)))

def display_mask(img):
    display(Image.fromarray(np.uint8(img*255)))

# Unzip


In [3]:
home_dir='/storage/BPI/Data/LNDST/'

In [9]:
result_files=glob.glob(home_dir+'*.npy')

In [10]:
result_files

['/storage/BPI/Data/LNDST/200901_0816_xresnet18_cv2_test.npy',
 '/storage/BPI/Data/LNDST/200901_0925_xresnet34_cv1_test.npy',
 '/storage/BPI/Data/LNDST/200901_0736_resnet18_cv2_test.npy',
 '/storage/BPI/Data/LNDST/200901_0749_xresnet18_cv0_test.npy',
 '/storage/BPI/Data/LNDST/200901_0803_xresnet18_cv1_test.npy',
 '/storage/BPI/Data/LNDST/200901_0830_resnet34_cv0_test.npy',
 '/storage/BPI/Data/LNDST/200901_0710_resnet18_cv0_test.npy',
 '/storage/BPI/Data/LNDST/200901_0844_resnet34_cv1_test.npy',
 '/storage/BPI/Data/LNDST/200901_0857_resnet34_cv2_test.npy',
 '/storage/BPI/Data/LNDST/200901_0723_resnet18_cv1_test.npy',
 '/storage/BPI/Data/LNDST/200901_0939_xresnet34_cv2_test.npy',
 '/storage/BPI/Data/LNDST/200901_0911_xresnet34_cv0_test.npy']

### Do each model individually

In [16]:
for result_file in result_files:
    preds=np.load(result_file)
    # convert from probability to binary mask and change dytpe per competition instructions
    preds=np.argmax(preds,1).astype('int8').flatten()
    
#     assert(2==3)

    # np.save(f'/content/{os.path.basename(result_file)[:-9]}_submission.npy',preds)
    np.save(f'{home_dir}Submissions/{os.path.basename(result_file)[:-9]}_submission.npy',preds)

### Do each architecture together

In [27]:
archs=['_resnet18_','_xresnet18_','_resnet34_','_xresnet34_']

In [45]:
for arch in archs:
    cur_result_files=[ii for ii in result_files if arch in ii]
    
    preds=np.load(cur_result_files[0])
    for result_file in cur_result_files[1:]:
        curpreds=np.load(result_file)
        
        preds[:,0,:,:]=preds[:,0,:,:]+curpreds[:,0,:,:]
        preds[:,1,:,:]=preds[:,1,:,:]+curpreds[:,1,:,:]
        
    preds=np.argmax(preds,1).astype('int8').flatten()
    np.save(f'{home_dir}Submissions/{arch}submission.npy',preds)

### Do each depth together

In [46]:
archs=['resnet18_','resnet34_']

In [48]:
for arch in archs:
    cur_result_files=[ii for ii in result_files if arch in ii]
    
#     print(cur_result_files)
    
    preds=np.load(cur_result_files[0])
    for result_file in cur_result_files[1:]:
        curpreds=np.load(result_file)
        
        preds[:,0,:,:]=preds[:,0,:,:]+curpreds[:,0,:,:]
        preds[:,1,:,:]=preds[:,1,:,:]+curpreds[:,1,:,:]
        
    preds=np.argmax(preds,1).astype('int8').flatten()
    np.save(f'{home_dir}Submissions/all_{arch}submission.npy',preds)

### Do all together

In [49]:
preds=np.load(result_files[0])
for result_file in result_files[1:]:
    curpreds=np.load(result_file)

    preds[:,0,:,:]=preds[:,0,:,:]+curpreds[:,0,:,:]
    preds[:,1,:,:]=preds[:,1,:,:]+curpreds[:,1,:,:]

preds=np.argmax(preds,1).astype('int8').flatten()
np.save(f'{home_dir}Submissions/all_submission.npy',preds)

### Do weighted average

In [56]:
archs=['resnet18_','resnet34_']

for weight in [1.1,1.2,1.5,2]:
    cur_result_files=[ii for ii in result_files if 'resnet18_' in ii]
    
    preds=np.load(cur_result_files[0])
    for result_file in cur_result_files[1:]:
        curpreds=np.load(result_file)
        
        preds[:,0,:,:]=preds[:,0,:,:]+curpreds[:,0,:,:]
        preds[:,1,:,:]=preds[:,1,:,:]+curpreds[:,1,:,:]
        
    cur_result_files=[ii for ii in result_files if 'resnet34_' in ii]    
    
    for result_file in cur_result_files:
        curpreds=np.load(result_file)
        
        preds[:,0,:,:]=preds[:,0,:,:]+curpreds[:,0,:,:]*weight
        preds[:,1,:,:]=preds[:,1,:,:]+curpreds[:,1,:,:]*weight
        
    preds=np.argmax(preds,1).astype('int8').flatten()
    
    
    wstr=str(weight).replace('.','_')
    np.save(f'{home_dir}Submissions/all_{wstr}{arch}_submission.npy',preds)